In [1]:
import re
import spacy
from fastcoref import spacy_component

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("fastcoref")  # Add coreference resolution component

def mask_double_quotes(text):
    """Replaces all types of double-quoted text (straight or curly) with placeholders."""
    # Matches text inside either "..." or “...”, non-greedy, across lines too
    quote_pattern = r'(?:(?:"|“)(.*?)(?:"|”))'
    placeholders = {}
    
    def replacer(match):
        full_quote = match.group(0)
        placeholder = f"__QUOTE_{len(placeholders)}__"
        placeholders[placeholder] = full_quote
        return placeholder
    
    masked_text = re.sub(quote_pattern, replacer, text, flags=re.DOTALL)
    return masked_text, placeholders

def unmask_double_quotes(text, placeholders):
    """Restores the original text inside double quotes."""
    for placeholder, original in placeholders.items():
        text = text.replace(placeholder, original)
    return text

def resolve_coreferences(text):
    # Step 1: Mask double-quoted text
    masked_text, placeholders = mask_double_quotes(text)

    # Step 2: Perform coreference resolution
    doc = nlp(masked_text, component_cfg={"fastcoref": {'resolve_text': True}})

    # Step 3: Get resolved text
    resolved_text = doc._.resolved_text

    # Step 4: Restore double-quoted text
    final_text = unmask_double_quotes(resolved_text, placeholders)
    
    return final_text

c:\Users\uzmap\anaconda3\envs\python310\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\uzmap\anaconda3\envs\python310\lib\site-packages\transformers\modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlis

In [2]:
from datasets import load_dataset
import pandas as pd

# Load the dataset
ds = load_dataset("honicky/short_childrens_stories_with_labeled_character_names")

# Convert the training set to a DataFrame
df_train = pd.DataFrame(ds['train'])
df_train.head()

,story,characters
0,"In a small town named Mathville, lived two be...","Algy,Sammy"
1,"One sunny day, Alex and Jamie decided to go f...","Alex,Jamie"
2,"In the hot, dusty town of Mississippi during ...","Bowie,Chicamaw,T-Dub,Mattie,Lula,Keechie"
3,"In the bustling city of Philadelphia, Pennsyl...","Timmy,Sarah,Jimmy Lyggett SR"
4,"It was a sunny day in Home Runville, a town f...","Tommy,Alexa,Mark"


In [5]:
# Resolve coreferences
resolved_texts = resolve_coreferences(df_train["story"][1550])

04/21/2025 20:01:44 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

04/21/2025 20:01:44 - INFO - 	 ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
resolved_texts

' In a small town named Chillyville, way up north in Alaska, lived two curious little seeds named Almond and Aloi. two curious little seeds named Almond and Aloi dreamed of becoming big and strong like the trees and flowers two curious little seeds named Almond and Aloi had heard about in two curious little seeds named Almond and Aloi\'s seed stories. One sunny day, two curious little seeds named Almond and Aloi decided to set out on an adventure to find two curious little seeds named Almond and Aloi\'s perfect home to grow.\n\nAs two curious little seeds named Almond and Aloi rolled along, Almond asked, "Have you ever wondered what kind of place would be best for us?" Aloi replied thoughtfully, "Well, I know we need warmth and water, so let\'s explore and see what we can find!"\n\ntwo curious little seeds named Almond and Aloi came across many beautiful places—shiny snowy mountains, deep blue lakes, and dense forests. But something didn\'t feel right; neither Almond nor Aloi felt two 